#Web Scraping by using The API of the web site www.faire.com

#Importing packages

In [ ]:
import requests
import json
import pandas
from pandas import json_normalize
import pandas as pd
import time

In [ ]:
headers = {
  'authority': 'cdn.cookielaw.org',
  'pragma': 'no-cache',
  'cache-control': 'no-cache',
  'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
  'sec-ch-ua-mobile': '?0',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
  'accept': '*/*',
  'origin': 'https://www.faire.com',
  'sec-fetch-site': 'cross-site',
  'sec-fetch-mode': 'cors',
  'sec-fetch-dest': 'empty',
  'referer': 'https://www.faire.com/',
  'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7'
}

In [ ]:
All_Brands = pd.DataFrame([])
url = "https://www.faire.com/api/brand/list-brands"
r = requests.get(url, headers=headers)
data = json.loads(r.text)
All_Brands = All_Brands.append(pandas.json_normalize(data['brands']),ignore_index=True)


In [ ]:
print('***************** ***************** *****************')
print('               we have ',len(All_Brands),'brands')
print('***************** ***************** *****************')
All_Brands.head()

***************** ***************** *****************
               we have  25267 brands
***************** ***************** *****************


,name,token
0,Craft a Brew,b_dd0d5049
1,Soberdough,b_e2c1b582
2,Click and Grow,b_61bab788
3,Let Them Eat Candles,b_5fa5a298
4,Hella Bitter,b_672cf373


In [ ]:
Brands = pd.DataFrame([])
Brands2 = pd.DataFrame([])
for j in range(0,10000):
  url1 = 'https://www.faire.com/api/brand-view/'+str(All_Brands.token[j])
  r = requests.get(url1, headers=headers)
  data1 = json.loads(r.text)
  Brands = Brands.append(pandas.json_normalize(data1['brand']),ignore_index=True)

  url2 = 'https://www.faire.com/api/brand/'+str(All_Brands.token[j])+'/top-category'
  r2 = requests.get(url2, headers=headers)
  data2 = json.loads(r2.text)
  Brands2 = Brands2.append(pandas.json_normalize(data2),ignore_index=True)
  time.sleep(0.05)

#Scrape Faire.com for brand names, countries, and 1-2 categories

In [ ]:
DF = pandas.concat([Brands['name'], Brands['made_in'],Brands['based_in_city'],Brands2['category_name'],Brands2['subcategory_name']],axis = 1)
DF

,name,made_in,based_in_city,category_name,subcategory_name
0,Craft a Brew,USA,Orlando,Food & Drink,DIY Kits
1,Soberdough,USA,Nashville,Food & Drink,Baking
2,Click and Grow,EST,San Francisco,Home Decor,Garden & Outdoor
3,Let Them Eat Candles,USA,Chicago,Food & Drink,Confections
4,Hella Bitter,USA,New York,Food & Drink,Beverages
...,...,...,...,...,...
9995,Shoji Note,USA,Portland,Paper & Novelty,Greeting Cards
9996,r e n o o || meditation goods for the modern...,USA,Brooklyn,Home Decor,Home Accents
9997,Putumayo,USA,New Orleans,Paper & Novelty,Greeting Cards
9998,XXIV Active,CHN,NaN,Women,Apparel


#Enrichement Faire.com brands with other relevant information

In [ ]:
DF1 = pandas.concat([All_Brands[:10000],Brands['description'],Brands['likes'],Brands['owner_first_name'],Brands['owner_last_name'],
                    Brands['made_in'],Brands['based_in_city'],Brands2['category_name'],Brands2['subcategory_name'],Brands['url'],Brands['video_url'],Brands['creation_year'],
                    Brands['facebook_handle'],Brands['facebook_followers'],Brands['instagram_handle'],Brands['instagram_followers'],
                    Brands['twitter_handle'],Brands['twitter_followers'],Brands['pinterest_handle'],Brands['pinterest_followers'],
                    Brands['active_products_count'],Brands['version'],Brands['estimated_ship_date_formatted'],Brands['currency'],
                    Brands['brand_reviews_summary.review_percentage_by_rating.1'],Brands['brand_reviews_summary.review_percentage_by_rating.2'],
                    Brands['brand_reviews_summary.review_percentage_by_rating.3'],Brands['brand_reviews_summary.review_percentage_by_rating.4'],
                    Brands['brand_reviews_summary.review_percentage_by_rating.5'],Brands['reorder_minimum_amount.amount_cents'],
                    Brands['marketing_email_subject']],axis = 1)
DF1

,name,token,description,likes,owner_first_name,owner_last_name,made_in,based_in_city,category_name,subcategory_name,url,video_url,creation_year,facebook_handle,facebook_followers,instagram_handle,instagram_followers,twitter_handle,twitter_followers,pinterest_handle,pinterest_followers,active_products_count,version,estimated_ship_date_formatted,currency,brand_reviews_summary.review_percentage_by_rating.1,brand_reviews_summary.review_percentage_by_rating.2,brand_reviews_summary.review_percentage_by_rating.3,brand_reviews_summary.review_percentage_by_rating.4,brand_reviews_summary.review_percentage_by_rating.5,reorder_minimum_amount.amount_cents,marketing_email_subject
0,Craft a Brew,b_dd0d5049,Here at Craft a Brew we’re on a mission to tur...,8388.0,Kyle,Westfall,USA,Orlando,Food & Drink,DIY Kits,https://craftabrew.com/,https://vimeo.com/78576857,2010.0,NaN,22070.0,craftabrew,8388.0,NaN,1698.0,NaN,NaN,37,6175,"Estimated Delivery Aug 7 - 9, 2021",USD,NaN,NaN,NaN,7.0,93.0,5000.0,NaN
1,Soberdough,b_e2c1b582,Home baked bread made simple. Just add beer. N...,2222.0,Soberdough,Team,USA,Nashville,Food & Drink,Baking,https://www.soberdough.com/,https://www.youtube.com/watch?v=o1hbZ3wzV5g,2013.0,Soberdough/,5454.0,soberdough,2222.0,NaN,561.0,NaN,NaN,27,6471,"Estimated Delivery Aug 7 - 9, 2021",USD,0.0,NaN,1.0,2.0,97.0,17000.0,Soberdough: artisan brew bread.
2,Click and Grow,b_61bab788,Click and Grow’s Smart Gardens are an easy and...,5520.0,Triin,Lokk,EST,San Francisco,Home Decor,Garden & Outdoor,https://www.clickandgrow.com/,https://www.youtube.com/watch?v=-7Hn-pWYBKw,2009.0,NaN,33831.0,clickandgrow,27553.0,clickandgrow,5520.0,clickandgrow,1166.0,20,5508,"Estimated Delivery Aug 14 - 16, 2021",USD,NaN,NaN,NaN,NaN,100.0,15000.0,Innovative Smart Indoor Garden just for YOU
3,Let Them Eat Candles,b_5fa5a298,Let Them Eat Candles was founded in 2012 by Lo...,550.0,Loree,Sandler,USA,Chicago,Food & Drink,Confections,http://www.letthemeatcandles.com/,https://www.youtube.com/watch?v=HqVEORIQags,2012.0,NaN,411.0,letthemeatcandles,550.0,NaN,NaN,NaN,NaN,7,2871,"Estimated Delivery Aug 21 - 27, 2021",USD,NaN,NaN,8.0,NaN,92.0,11400.0,"Have your cake and EAT YOUR CANDLES, too!"
4,Hella Bitter,b_672cf373,Hella Cocktail Co. is a New York City-based co...,25147.0,Jomaree,Pinkard,USA,New York,Food & Drink,Beverages,http://hellacocktail.co/,NaN,2011.0,NaN,NaN,hellacocktailco,25147.0,NaN,NaN,NaN,NaN,20,10736,"Estimated Delivery Aug 8 - 10, 2021",USD,6.0,6.0,NaN,6.0,83.0,0.0,No Bar Is Complete Without A Handcrafted Hella...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Shoji Note,b_1bxgdbkwrh,Sloan Fisher is a graphic designer and craftsp...,1287.0,Sloan,Fisher,USA,Portland,Paper & Novelty,Greeting Cards,https://shojinote.com/,NaN,2016.0,NaN,NaN,shojinote,1287.0,NaN,NaN,NaN,NaN,115,2692,"Estimated Delivery Aug 10 - 13, 2021",USD,NaN,NaN,NaN,NaN,NaN,10000.0,Greeting cards lovingly handcrafted by me for ...
9996,r e n o o || meditation goods for the modern...,b_auj8kxcemh,"At renoo, we build our meditation products aro...",5.0,r e n o o,- meditation goods for the modern home,USA,Brooklyn,Home Decor,Home Accents,http://www.renoo.life/,NaN,2018.0,renoolife,92.0,renoolife,4815.0,renoolife,5.0,renoolife,6.0,74,4642,"Estimated Delivery Aug 7 - 9, 2021",USD,NaN,NaN,NaN,NaN,100.0,10000.0,Mindfully-made meditation goods for the modern...
9997,Putumayo,b_1iwc2eoa3q,Putumayo began in 1975 as a retailer and later...,NaN,Dan,Storper,USA,New Orleans,Paper & Novelty,Greeting Cards,http://putumayo.com/greeting-cards,NaN,1993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51,2742,"Estimated Delivery Aug 7 - 9, 2021",USD,NaN,NaN,NaN,NaN,100.0,10000.0,World art and photography greeting cards guara...
9998,XXIV Active,b_8hdya19ivb,Our mission is to empower you to fullfill your...,23904.0,David,Tou,CHN,NaN,Women,Apparel,https://xxivactive.com/,NaN,2017.0,NaN,NaN,xxivactive,23904.0,NaN,NaN,NaN,NaN,27,3515,"

In [ ]:
DF1.to_csv("Faire_Brands.xlsx",sep=';', encoding='utf-8')